# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

<u>Librerias that will be used in the project are:</u>
- **pandas** for data stracture and data analysis 
- **nltk** for categorization 

The df is loaded with try-except method for non local users 

In [1]:
# Loading all the libraries
import pandas as pd 
import nltk 
from nltk.stem import WordNetLemmatizer 
wordnet_lemma = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
temmer=PorterStemmer()

# Load the data
try:
    df= pd.read_csv(r'C:/Users/wolff/Downloads/credit_scoring_eng.csv')
except:
    df= pd.read_csv('/datasets/credit_scoring_eng.csv')
    

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [2]:
# Let's see how many rows and columns our dataset has
df.shape

(21525, 12)

Using the shape method we see we have 21525 rows and 12 columns 
***

In [3]:
# let's print the first N rows
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


- days_employed with negitive days might be an issue needs to be explored 
***

In [4]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


 - columns **'days_employed'** and **'total_income'** have the same count of  missing values 
***

In [5]:
# Let's look in the filtered table at the the first column with missing data
df[df.days_employed.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


We see that where we have **'days_employed'** missing values we see missing values in **total_income**.

My assumption is that the missing values sample are same size for both columns and they are linkd, simply put we can't have Income without days_employed.
***

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
print((df['days_employed'].isna().sum()) & (df['total_income'].isna().sum()))

2174


In [7]:
(df.isna().sum()/len(df))*100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [8]:
na_values = df.isna().sum().to_frame('missing_values')
na_values['%'] = round(df.isna().sum()/len(df),3)
na_values.sort_values(by='%', ascending=False)

,missing_values,%
days_employed,2174,0.101
total_income,2174,0.101
children,0,0.000
dob_years,0,0.000
education,0,0.000
education_id,0,0.000
family_status,0,0.000
family_status_id,0,0.000
gender,0,0.000
income_type,0,0.000


<u>**Intermediate conclusion**</u>

The number of rows in filterd table is a match for the number of missing values, we can conclude that the filtered table is the source of our missing values, and that there is symmetry in the missing values. 

The missing value is 10% of the total data, it is some what substantial and might need to be filled. 

My next step is to check specific client characteristic - income_type that the missing values have dependenceis on.
***

In [9]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
df[df.days_employed.isnull()]['income_type'].value_counts()
print('----------------------------------------------------')
df[df.days_employed.isnull()]['income_type'].value_counts().sum()

----------------------------------------------------


2174

In [10]:
# Checking distribution
print((df['income_type'].value_counts()/len(df))*100)

employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
entrepreneur                    0.009292
unemployed                      0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64


Considering whether the missing data could be due to the specific client characteristic.
Income type has 2174 missing values for days_employed

**Possible reasons for missing values in data**

After Checking Income_type i see that the missing values does not come from a specific line (specific income_type), rather its across few lines.

this suggests randomness


In [11]:
# Checking the distribution in the whole dataset
for value in df:
    print(df[df.days_employed.isnull()][value].value_counts())
    print('----------------------------------------')

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64
----------------------------------------
Series([], Name: days_employed, dtype: int64)
----------------------------------------
34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
69     5
20     5
71     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64
----------------------------------------
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACH

**Conclusions**

The original dataset with out the filtering gave us randon distrabutain same as filtered table.

The missing values are random, in income_type there is not one specific line that is the couse of the missing values.

It seems that all of the missing values are random across ages, education, income_type and so on. 

I will will use income type to calculate means and fill the missing values with them in the following steps 

***

## Data transformation


In [12]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [13]:
# Fix the registers if required
df['education'] = df['education'].str.lower()

<div class="alert alert-success" role="alert">
<b>Reviewer's comment v1</b>

Great that you have applied  `.str.lower()` to convert string data to the same lowercase format.

</div>

In [14]:
# Checking all the values in the column to make sure we fixed them
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

***

In [15]:
# Let's see the distribution of values in the `children` column
(df['children'].value_counts()/len(df))*100

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

There is -1 for amout of children it is 0.2% of the data and 20 for amout of children it is 0.35% , most likly a errors in typing/importing data and it should be 1 and 2 accordingly.


In [16]:
# [fix the data based on your decision]
df['children'] = df['children'].replace(-1,1)
df['children'] = df['children'].replace(20,2)

<div class="alert alert-success" role="alert">
<b>Reviewer's comment v1</b>
    
Well done! These values have been indeed suspicious. 


In [17]:
# Checking the `children` column again to make sure it's all fixed
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

***

In [18]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
(df['days_employed'] < 0).value_counts()/len(df['days_employed'])*100

True     73.89547
False    26.10453
Name: days_employed, dtype: float64

We can see that values that are below 0 in days_employed are ~74% this very high, we cant work negtive days. its most likely due to a worng calculations or data imput.

In [19]:
# Address the problematic values, if they exist
df['days_employed'] = abs(df['days_employed'])

In [20]:
# Check the result - make sure it's fixed
(df['days_employed'] < 0).value_counts()/len(df['days_employed'])*100

False    100.0
Name: days_employed, dtype: float64

***

In [21]:
# Check the `dob_years` for suspicious values and count the percentage
((df['dob_years'].value_counts()/len(df))*100).sort_values()

75    0.004646
74    0.027875
73    0.037166
19    0.065041
72    0.153310
20    0.236934
71    0.269454
70    0.301974
69    0.394890
68    0.459930
0     0.469222
21    0.515679
67    0.775842
22    0.850174
66    0.850174
65    0.901278
23    1.180023
24    1.226481
64    1.231127
63    1.249710
62    1.635308
61    1.649245
25    1.658537
60    1.751452
26    1.895470
55    2.058072
59    2.062718
51    2.081301
53    2.132404
57    2.137050
58    2.141696
46    2.206736
54    2.225319
47    2.229965
52    2.248548
56    2.262485
27    2.290360
45    2.308943
28    2.336818
49    2.360046
32    2.369338
43    2.383275
50    2.387921
37    2.494774
48    2.499419
30    2.508711
29    2.531940
44    2.541231
36    2.578397
31    2.601626
39    2.662021
33    2.699187
42    2.773519
38    2.778165
34    2.801394
41    2.819977
40    2.829268
35    2.866434
Name: dob_years, dtype: float64

We have 0 in age, this is ~0.5% of the age column. This will replace this with a mean of age 

In [22]:
# Address the issues in the `dob_years` column, if they exist
df['dob_years'].mean()
df['dob_years'] = df['dob_years'].replace(0,df['dob_years'].mean())

In [23]:
# Check the result - make sure it's fixed
((df['dob_years'].value_counts()/len(df))*100).sort_values()

75.00000    0.004646
74.00000    0.027875
73.00000    0.037166
19.00000    0.065041
72.00000    0.153310
20.00000    0.236934
71.00000    0.269454
70.00000    0.301974
69.00000    0.394890
68.00000    0.459930
43.29338    0.469222
21.00000    0.515679
67.00000    0.775842
66.00000    0.850174
22.00000    0.850174
65.00000    0.901278
23.00000    1.180023
24.00000    1.226481
64.00000    1.231127
63.00000    1.249710
62.00000    1.635308
61.00000    1.649245
25.00000    1.658537
60.00000    1.751452
26.00000    1.895470
55.00000    2.058072
59.00000    2.062718
51.00000    2.081301
53.00000    2.132404
57.00000    2.137050
58.00000    2.141696
46.00000    2.206736
54.00000    2.225319
47.00000    2.229965
52.00000    2.248548
56.00000    2.262485
27.00000    2.290360
45.00000    2.308943
28.00000    2.336818
49.00000    2.360046
32.00000    2.369338
43.00000    2.383275
50.00000    2.387921
37.00000    2.494774
48.00000    2.499419
30.00000    2.508711
29.00000    2.531940
44.00000    2

***

In [24]:
# Let's see the values for the column
((df['family_status'].value_counts()/len(df))*100)

married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64

In [25]:
# Address the problematic values in `family_status`, if they exist

In [26]:
# Check the result - make sure it's fixed

No issues fot 'family_status' column
***

In [27]:
# Let's see the values in the column
((df['gender'].value_counts()/len(df))*100)

F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64

In [28]:
# Address the problematic values, if they exist
df = df[df.gender != 'XNA']

In [29]:
# Check the result - make sure it's fixed
((df['gender'].value_counts()/len(df))*100)

F    66.140123
M    33.859877
Name: gender, dtype: float64

Small precentage of the gender column data is 'XNA' i decided to remove it, although it may not have relevance for analysis
***

In [30]:
# Let's see the values in the column
((df['family_status'].value_counts()/len(df))*100)

married              57.517190
civil partnership    19.401598
unmarried            13.069132
divorced              5.551942
widow / widower       4.460138
Name: family_status, dtype: float64

In [31]:
# Address the problematic values, if they exist

In [32]:
# Check the result - make sure it's fixed

No issues fot 'family_status' column
***

In [33]:
# Checking duplicates
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [34]:
# Address the duplicates, if they exist
stemmer = PorterStemmer()

def type_purpose(purpose):
    for word in purpose.split(' '):
        if 'car' == stemmer.stem(word):
            return 'car purchase'
        if 'wed' == stemmer.stem(word):
            return 'wedding expenses'
        if ('educ' == stemmer.stem(word) or 'univers' == stemmer.stem(word)):
            return 'education expenses'
        if ('hous' == stemmer.stem(word) or 'estat' == stemmer.stem(word) or 'properti' == stemmer.stem(word)):
            return 'house purchase'
    return purpose

df['purpose'] = df['purpose'].apply(type_purpose)

In [35]:
# Last check whether we have any duplicates
df['purpose'].unique()

array(['house purchase', 'car purchase', 'education expenses',
       'wedding expenses'], dtype=object)

I used PorterStemmer to remove duplicates in 'purpose' column and to categorize
***

In [36]:
# Check the size of the dataset that you now have after your first manipulations with it
df['purpose'].value_counts().sum()

21524

In [37]:
(df['purpose'].value_counts()/len(df))*100

house purchase        50.357740
car purchase          20.047389
education expenses    18.686118
wedding expenses      10.908753
Name: purpose, dtype: float64

Removed duplicates with the same meaning that was writing in a diffrent way.

Most of the reqestes for loans are for Housing and car purchases

***

In [38]:
df.duplicated().sum()

405

In [39]:
df.drop_duplicates(inplace=True)

In [40]:
df.duplicated().sum()

0

# Working with missing values

Creating dictionaries for **total_income** and **days_employed**


In [41]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [42]:
# Find the dictionaries
grouped_income = df.groupby(['income_type'])['total_income'].median()
grouped_days = df.groupby(['income_type'])['days_employed'].mean()

grouped_income
print('---------------------------------------------------')
grouped_days

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

---------------------------------------------------


income_type
business                         2111.470404
civil servant                    3399.896902
employee                         2326.499216
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365003.491245
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [43]:
grouped_income_dict=pd.Series(grouped_income).to_dict()
grouped_days_dict=pd.Series(grouped_days).to_dict()

grouped_income_dict
print('---------------------------------------------------')
grouped_days_dict

{'business': 27571.0825,
 'civil servant': 24071.6695,
 'employee': 22815.103499999997,
 'entrepreneur': 79866.103,
 'paternity / maternity leave': 8612.661,
 'retiree': 18962.318,
 'student': 15712.26,
 'unemployed': 21014.360500000003}

---------------------------------------------------


{'business': 2111.470404393895,
 'civil servant': 3399.896901695746,
 'employee': 2326.4992159718063,
 'entrepreneur': 520.8480834953765,
 'paternity / maternity leave': 3296.7599620220594,
 'retiree': 365003.49124486075,
 'student': 578.7515535382181,
 'unemployed': 366413.65274420456}

In [44]:
grouped_income_dict.values()
print('---------------------------------------------------')
grouped_days_dict.values()

dict_values([27571.0825, 24071.6695, 22815.103499999997, 79866.103, 8612.661, 18962.318, 15712.26, 21014.360500000003])

---------------------------------------------------


dict_values([2111.470404393895, 3399.896901695746, 2326.4992159718063, 520.8480834953765, 3296.7599620220594, 365003.49124486075, 578.7515535382181, 366413.65274420456])

In [45]:
grouped_income_dict.keys()
print('---------------------------------------------------')
grouped_days_dict.keys()

dict_keys(['business', 'civil servant', 'employee', 'entrepreneur', 'paternity / maternity leave', 'retiree', 'student', 'unemployed'])

---------------------------------------------------


dict_keys(['business', 'civil servant', 'employee', 'entrepreneur', 'paternity / maternity leave', 'retiree', 'student', 'unemployed'])

In [46]:
grouped_income_dict
print('---------------------------------------------------')
grouped_days_dict

{'business': 27571.0825,
 'civil servant': 24071.6695,
 'employee': 22815.103499999997,
 'entrepreneur': 79866.103,
 'paternity / maternity leave': 8612.661,
 'retiree': 18962.318,
 'student': 15712.26,
 'unemployed': 21014.360500000003}

---------------------------------------------------


{'business': 2111.470404393895,
 'civil servant': 3399.896901695746,
 'employee': 2326.4992159718063,
 'entrepreneur': 520.8480834953765,
 'paternity / maternity leave': 3296.7599620220594,
 'retiree': 365003.49124486075,
 'student': 578.7515535382181,
 'unemployed': 366413.65274420456}

***

### Restoring missing values in `total_income`

I have missing data in total_income and days_employed i will reaplce the data through dictionary i have created

In [47]:
print(df['dob_years'].value_counts())
print(df['dob_years'].max())
print(df['dob_years'].min())

35.00000    605
40.00000    600
34.00000    599
41.00000    599
38.00000    592
42.00000    583
33.00000    572
39.00000    563
31.00000    555
36.00000    547
29.00000    539
44.00000    538
37.00000    534
30.00000    530
48.00000    529
32.00000    506
43.00000    504
50.00000    504
28.00000    498
49.00000    494
45.00000    488
27.00000    488
52.00000    473
56.00000    471
46.00000    468
54.00000    464
47.00000    460
53.00000    450
57.00000    445
58.00000    442
51.00000    439
59.00000    438
55.00000    435
26.00000    403
60.00000    363
25.00000    357
61.00000    346
62.00000    340
24.00000    263
63.00000    262
64.00000    250
23.00000    244
65.00000    189
22.00000    183
66.00000    177
67.00000    164
21.00000    110
43.29338    100
68.00000     98
69.00000     85
70.00000     65
71.00000     55
20.00000     51
72.00000     33
19.00000     14
73.00000      8
74.00000      6
75.00000      1
Name: dob_years, dtype: int64
75.0
19.0


In [48]:
# Let's write a function that calculates the age category 
def dob_group(dob_years):
    
    """
    The function returns the age group according to the age value, using the following rules:
    —'adult' for 19 <= age <= 64
    —'retired' for all other cases
    
    """
    if dob_years <= 64:
        return 'adult'
    return 'retired' 


In [49]:
# Test if the function works
print(dob_group(25)) 
print(dob_group(70)) 

adult
retired


In [50]:
# Creating new column based on function
df['dob_group'] = df['dob_years'].apply(dob_group)

In [51]:
# Checking how values in the new column
df['dob_group']

0          adult
1          adult
2          adult
3          adult
4          adult
          ...   
21520      adult
21521    retired
21522      adult
21523      adult
21524      adult
Name: dob_group, Length: 21119, dtype: object

In [52]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
df[df.notna()].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,house purchase,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,house purchase,adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education expenses,adult
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding expenses,adult


***

In [53]:
# Look at the mean values for income based on your identified factors
df.groupby(['income_type'])['total_income'].mean()
print('------------------------------------------')
df.groupby(['income_type'])['total_income'].median()

income_type
business                       32386.741818
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

------------------------------------------


income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [54]:
# Look at the median values for income based on your identified factors
df.groupby(['dob_group'])['total_income'].mean()
print('------------------------------------------')
df.groupby(['dob_group'])['total_income'].median()

dob_group
adult      27018.763347
retired    21542.650450
Name: total_income, dtype: float64

------------------------------------------


dob_group
adult      23382.688
retired    18471.391
Name: total_income, dtype: float64

In [55]:
df.groupby(['education'])['total_income'].mean()
print('------------------------------------------')
df.groupby(['education'])['total_income'].median()

education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29040.132990
Name: total_income, dtype: float64

------------------------------------------


education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25608.7945
Name: total_income, dtype: float64

<u>Repeated on sevral factors:</u>
- 'income_type' - students does not earn the same as entrepreneur
- 'dob_group' - retired dont work hence earn less  
- 'education' - can detrmain earning potential 

***

<u>Identified factor that is used:</u>

Income_type will be used as it will most determine the total_income as it compers more accurately salary.

I will use median to avoid outliers. 

I will use a dictionary i created to replce the missing values

***

In [56]:
#  Write a function that we will use for filling in missing values
df['total_income']=df['total_income'].fillna(df.income_type.map(grouped_income_dict))

In [57]:
# Check if it works
df['total_income'].isna().sum()

0

In [58]:
# Checking the number of entries in the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21119 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21119 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21119 non-null  float64
 3   education         21119 non-null  object 
 4   education_id      21119 non-null  int64  
 5   family_status     21119 non-null  object 
 6   family_status_id  21119 non-null  int64  
 7   gender            21119 non-null  object 
 8   income_type       21119 non-null  object 
 9   debt              21119 non-null  int64  
 10  total_income      21119 non-null  float64
 11  purpose           21119 non-null  object 
 12  dob_group         21119 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


days_employed still has missing values, will apply same method through dictionary to fill.
***

###  Restoring values in `days_employed`

In [59]:
# Distribution of `days_employed` medians based on your identified parameters
df.groupby(['income_type'])['days_employed'].mean()
print('------------------------------------------')
df.groupby(['income_type'])['days_employed'].median()

income_type
business                         2111.470404
civil servant                    3399.896902
employee                         2326.499216
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365003.491245
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

------------------------------------------


income_type
business                         1546.333214
civil servant                    2689.368353
employee                         1574.202821
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365213.306266
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [60]:
# Distribution of `days_employed` means based on your identified parameters

df.groupby(['dob_group'])['days_employed'].mean()
print('------------------------------------------')
df.groupby(['dob_group'])['days_employed'].median()

dob_group
adult       56008.347074
retired    314080.528722
Name: days_employed, dtype: float64

------------------------------------------


dob_group
adult        2049.685470
retired    360304.232308
Name: days_employed, dtype: float64

I will use days_employed by total_income and will use median to avoid outliers

In [61]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
df['days_employed']=df['days_employed'].fillna(df.income_type.map(grouped_days_dict))

In [62]:
# Check that the function works
df['days_employed'].isna().sum()

0

In [63]:
# Check the entries in all columns - make sure we fixed all missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21119 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21119 non-null  int64  
 1   days_employed     21119 non-null  float64
 2   dob_years         21119 non-null  float64
 3   education         21119 non-null  object 
 4   education_id      21119 non-null  int64  
 5   family_status     21119 non-null  object 
 6   family_status_id  21119 non-null  int64  
 7   gender            21119 non-null  object 
 8   income_type       21119 non-null  object 
 9   debt              21119 non-null  int64  
 10  total_income      21119 non-null  float64
 11  purpose           21119 non-null  object 
 12  dob_group         21119 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


All missing Values are handeled in the dataset 
***

## Categorization of data

In [64]:
# Print the values for your selected data for categorization
df['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21119, dtype: float64

In [65]:
# Check the unique values
df['total_income'].unique()
df['total_income'].max()
df['total_income'].min()

array([40620.102, 17932.802, 23341.752, ..., 14347.61 , 39054.888,
       13127.587])

362496.645

3306.762

In [66]:
# Let's write a function to categorize the data based on common topics
def income_level(total_income):    
    
    """
    The function returns the income group according to the income value, using the following rules:
    —'50K and below' for income <= 50000
    —'50K to 100K' for 50001 <= income <= 100000
    —'100K to 150K' for 101001 <= income <= 150000
    —'150K to 200K' for 150001 <= income <= 200000
    —'200K to 250K' for 200001 <= income <= 250000
    —'250K to 300K' for 250001 <= income <= 300000
    —'above 300K' for income >= 300001 
    
    """
        
    if total_income <= 10000:
        return '10K and below'
    if  10001 < total_income <= 20000:
        return '10K to 20K'
    if  20001 < total_income <= 30000:
        return '20K to 30K'
    if 30001 <= total_income <= 40000:
        return '30K to 40K'
    if 40001 <= total_income <= 50000:
        return '40K to 50K'
    if total_income < 50000:
        return 'Above 50'
            


df['income_level'] = df['total_income'].apply(income_level)
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_group,income_level
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,house purchase,adult,40K to 50K
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,10K to 20K
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,house purchase,adult,20K to 30K
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education expenses,adult,40K to 50K
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding expenses,adult,20K to 30K
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,house purchase,adult,40K to 50K
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,house purchase,adult,30K to 40K
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education expenses,adult,20K to 30K
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding expenses,adult,10K to 20K
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,house purchase,adult,20K to 30K


In [67]:
# Create a column with the categories and count the values for them
df['income_level'].value_counts()

20K to 30K       7541
10K to 20K       6731
30K to 40K       3106
40K to 50K       1492
10K and below     926
Above 50            2
Name: income_level, dtype: int64

In [68]:
# Looking through all the numerical data in your selected column for categorization

In [69]:
# Getting summary statistics for the column
df['income_level'].describe(include='all')

count          19798
unique             6
top       20K to 30K
freq            7541
Name: income_level, dtype: object

[Decide what ranges you will use for grouping and explain why.]

In [70]:
# Creating function for categorizing into different numerical groups based on ranges

In [71]:
# Creating column with categories

In [72]:
# Count each categories values to see the distribution

***

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [73]:
# Check the children data and paying back on time
df.groupby(['children'])['debt'].sum()
# Calculating default-rate based on the number of children
df.groupby(['children'])['debt'].mean().reset_index().sort_values('debt')
print('--------------------------------------------------------------------')
'Overall mean'
df['debt'].sum()/df['debt'].count()

children
0    1061
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

,children,debt
5,5,0.000000
0,0,0.076728
3,3,0.082067
1,1,0.092747
2,2,0.095508
4,4,0.100000


--------------------------------------------------------------------


'Overall mean'

0.08234291396372935

In [83]:
df.groupby('children')['debt'].agg(Count='count', Sum='sum', Mean = 'mean').reset_index()

,children,Count,Sum,Mean
0,0,13828,1061,0.076728
1,1,4798,445,0.092747
2,2,2115,202,0.095508
3,3,329,27,0.082067
4,4,40,4,0.100000
5,5,9,0,0.000000


**Conclusion**

The likelihood of loan will not be repaid on time does not increases with amount of children, hence there is no correlation

***

**Is there a correlation between family status and paying back on time?**

In [74]:
# Check the family status data and paying back on time
df.groupby(['family_status'])['debt'].sum()
# Calculating default-rate based on family status
df.groupby(['family_status'])['debt'].mean().reset_index().sort_values('debt')
print('--------------------------------------------------------------------')
'Overall mean'
df['debt'].sum()/df['debt'].count()

family_status
civil partnership    388
divorced              85
married              929
unmarried            274
widow / widower       63
Name: debt, dtype: int64

,family_status,debt
4,widow / widower,0.066808
1,divorced,0.071249
2,married,0.076929
0,civil partnership,0.094106
3,unmarried,0.098420


--------------------------------------------------------------------


'Overall mean'

0.08234291396372935

**Conclusion**

The likelihood of loan will not be repaid on time is thw worst for unmarried and for widow / widower and divorced can be said they have less loans that are overdue

***

**Is there a correlation between income level and paying back on time?**

In [75]:
# Check the income level data and paying back on time

df.groupby(['income_level'])['debt'].sum()
# Calculating default-rate based on income level
df.groupby(['income_level'])['debt'].mean().reset_index().sort_values('income_level')
print('--------------------------------------------------------------------')
'Overall mean'
df['debt'].sum()/df['debt'].count()


income_level
10K and below     58
10K to 20K       585
20K to 30K       660
30K to 40K       242
40K to 50K       102
Above 50           0
Name: debt, dtype: int64

,income_level,debt
0,10K and below,0.062635
1,10K to 20K,0.086911
2,20K to 30K,0.087522
3,30K to 40K,0.077914
4,40K to 50K,0.068365
5,Above 50,0.000000


--------------------------------------------------------------------


'Overall mean'

0.08234291396372935

In [76]:
income_pivot = pd.pivot_table(df, index = 'income_level', values = 'debt',aggfunc = 'mean')
print(income_pivot)

                   debt
income_level           
10K and below  0.062635
10K to 20K     0.086911
20K to 30K     0.087522
30K to 40K     0.077914
40K to 50K     0.068365
Above 50       0.000000


**Conclusion**

The likelihood of loan will not be repaid on time is the worst for borrowers with income level of **10K-20K and 20K - 30K** and the best for borrowers with 40K and above. 

***

**How does credit purpose affect the default rate?**

In [77]:
# Check the percentages for default rate for each credit purpose and analyze them
df.groupby(['purpose'])['debt'].sum()
# Calculating default-rate based on income level
df.groupby(['purpose'])['debt'].mean().reset_index().sort_values('debt')
print('--------------------------------------------------------------------')
'Overall mean'
df['debt'].sum()/df['debt'].count()

purpose
car purchase          402
education expenses    370
house purchase        781
wedding expenses      186
Name: debt, dtype: int64

,purpose,debt
2,house purchase,0.073839
3,wedding expenses,0.080659
1,education expenses,0.093340
0,car purchase,0.094101


--------------------------------------------------------------------


'Overall mean'

0.08234291396372935

**Conclusion**

The likelihood of loan will not be repaid on time is the worst for borrowers that are borrowing mony for a car purchase 
and the best for house purchase

***


# General Conclusion 

we can conclude that the likelihood of loan will not be repaid is higer for unmarried or civil partnership couples with income level of 10K-30K that want to make a car purchase.

In contrast  the likelihood of a loan will be repaid on time is higer for widow / widower or divorced persone that thier income level is 40K and above that want to make a house purchase


<u>What was done in order to get to a Conclusion:</u>

- Looking first at the data we saw missing_values for income_type and for days_employed:
    
    - the missing data is random 
    - specific client characteristic is income_type that the missing values have dependenceis on
    - For filling the missing data i used Method of dictionary and replaced the values with mean and median
    
- Duplicats were in education and for purpose
    - education solved by str.lower() method 
    - for purpose i used PorterStemmer() to creat funcation that finds words stems that i defined and returne catgory 
      here i combined categorization.
    
- Specials issued toticed: 
    - days_employed	 column most of values were in negtive, most likely due to worn impute or calculation fixed with abs()
    - for gender column we had 'XNA' gender witch most likely impute error i removed this rows - 47 in number
    - age column had rows with 0 in the age -> was replaced with mean() method  
    - children column had negtive amount in a few rows and 20 children a some rows -> replaced with 1 and 2 accordingly
    
- Categorization of data: 
    - for purpose i already done that previously when spoted the duplicates 
    - for total_income was done at the end the beeter answer 'How does credit purpose affect the default rate'